# 1. Download Data

This notebook downloads the necessary example data that will be used in other notebooks. In particular, the notebook does the following:

- Download beer and urine .mzML files used as examples in the paper
- Download the HMDB database and extract metabolites.
- Trains kernel density estimators on the mzML files.
- Extract regions of interests from the mzML files.

**Please run this notebook first to make sure the data files are available for subsequent notebooks.**

In [1]:
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path
import glob

In [4]:
import sys
sys.path.append('..')

In [5]:
from vimms.DataGenerator import extract_hmdb_metabolite, get_data_source, get_spectral_feature_database
from vimms.MassSpec import IndependentMassSpectrometer
from vimms.Controller import SimpleMs1Controller
from vimms.Common import *
from vimms.Roi import make_roi, RoiToChemicalCreator, extract_roi

In [6]:
# set_log_level_info()
set_log_level_debug()

## a. Download beer and urine files

Here we download the beer and urine .mzML files used as examples in the paper if they don't exist.

In [7]:
url = 'http://researchdata.gla.ac.uk/870/2/example_data.zip'
base_dir = os.path.join(os.getcwd(), 'example_data')

In [8]:
if not os.path.isdir(base_dir): # if not exist then download the example data and extract it
    print('Creating %s' % base_dir)    
    out_file = 'example_data.zip'
    download_file(url, out_file)
    extract_zip_file(out_file, delete=True)
else:
    print('Found %s' % base_dir)

Creating C:\Users\joewa\Work\git\vimms\examples\example_data


869kKB [02:17, 6.31kKB/s]                                                                                            


Extracting example_data.zip


100%|██████████████████████████████████████████████████████████████████████████████| 110/110 [00:10<00:00, 10.60it/s]


Deleting example_data.zip


## b. Download metabolites from HMDB

Next we load a pre-processed pickled file of database metabolites in the `data_dir` folder. If it is not found, then create the file by downloading and extracting the metabolites from HMDB.

In [9]:
compound_file = Path(base_dir, 'hmdb_compounds.p')
hmdb_compounds = load_obj(compound_file)
if hmdb_compounds is None: # if file does not exist

    # download the entire HMDB metabolite database, big and slow!!
    # url = 'http://www.hmdb.ca/system/downloads/current/hmdb_metabolites.zip'

    # download a smaller urine metabolite database for testing
    url = 'http://www.hmdb.ca/system/downloads/current/urine_metabolites.zip'

    out_file = download_file(url)
    compounds = extract_hmdb_metabolite(out_file, delete=True)
    save_obj(compounds, compound_file)

else:
    print('Loaded %d DatabaseCompounds from %s' % (len(hmdb_compounds), compound_file))

Old, invalid or missing pickle in C:\Users\joewa\Work\git\vimms\examples\example_data\hmdb_compounds.p. Please regenerate this file.


26.7kKB [00:04, 6.39kKB/s]                                                                                           


Extracting HMDB metabolites from urine_metabolites.zip
Loaded 4236 DatabaseCompounds from urine_metabolites.zip
Deleting urine_metabolites.zip
Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\hmdb_compounds.p


## c. Train the KDEs

In this section we demonstrate how ViMMS can be used to train kernel density estimators (KDEs) on the example Beer mzML files. The KDEs will be used to sample the MS1/MS2 data for chemicals, scan durations and number of peaks during simulation.

The following two methods `get_data_source` and `get_spectral_feature_database` from ViMMS will be used. 
- `get_data_source` loads a `DataSource` object that stores information on a set of .mzML files
- `get_spectral_feature_database` extracts relevant features from .mzML files that have been loaded into the DataSource. 

The parameter below should work for most cases, however for different data, it might be necessary to adjust the `min_rt` and `max_rt` values.

In [10]:
filename = None                    # if None, use all mzML files found
min_ms1_intensity = 0              # min MS1 intensity threshold to include a data point for density estimation
min_ms2_intensity = 0              # min MS2 intensity threshold to include a data point for density estimation
min_rt = 0                         # min RT to include a data point for density estimation
max_rt = 1440                      # max RT to include a data point for density estimation
bandwidth_mz_intensity_rt = 1.0    # kernel bandwidth parameter to sample (mz, RT, intensity) values during simulation
bandwidth_n_peaks = 1.0            # kernel bandwidth parameter to sample number of peaks per scan during simulation

### Load fullscan data and train KDEs

In [11]:
mzml_path = Path(base_dir, 'beers', 'fullscan', 'mzML')
xcms_output = Path(mzml_path, 'extracted_peaks_ms1.csv')
out_file = Path(base_dir, 'peak_sampler_mz_rt_int_19_beers_fullscan.p')

In [12]:
ds_fullscan = get_data_source(mzml_path, filename, xcms_output)

INFO   : DataSource                     : Loading Beer_multibeers_10_fullscan1.mzML
INFO   : DataSource                     : Loading Beer_multibeers_11_fullscan1.mzML
INFO   : DataSource                     : Loading Beer_multibeers_12_fullscan1.mzML
INFO   : DataSource                     : Loading Beer_multibeers_13_fullscan1.mzML
INFO   : DataSource                     : Loading Beer_multibeers_14_fullscan1.mzML
INFO   : DataSource                     : Loading Beer_multibeers_15_fullscan1.mzML
INFO   : DataSource                     : Loading Beer_multibeers_16_fullscan1.mzML
INFO   : DataSource                     : Loading Beer_multibeers_17_fullscan1.mzML
INFO   : DataSource                     : Loading Beer_multibeers_18_fullscan1.mzML
INFO   : DataSource                     : Loading Beer_multibeers_19_fullscan1.mzML
INFO   : DataSource                     : Loading Beer_multibeers_1_fullscan1.mzML
INFO   : DataSource                     : Loading Beer_multibeers_2_fullscan1

In [13]:
ps = get_spectral_feature_database(ds_fullscan, filename, min_ms1_intensity, min_ms2_intensity, min_rt, max_rt,
               bandwidth_mz_intensity_rt, bandwidth_n_peaks, out_file)

DEBUG  : PeakSampler                    : Extracted 0 MS2 scans
DEBUG  : PeakSampler                    : Computing parent intensity proportions
DEBUG  : PeakSampler                    : Extracting scan durations
DEBUG  : PeakSampler                    : Training KDEs for ms_level=1
DEBUG  : PeakSampler                    : Retrieving mz_intensity_rt values from <vimms.DataGenerator.DataSource object at 0x00000219E68862E8>
INFO   : DataSource                     : Using values from XCMS peaklist
DEBUG  : PeakSampler                    : Retrieving n_peaks values from <vimms.DataGenerator.DataSource object at 0x00000219E68862E8>
DEBUG  : PeakSampler                    : Training KDEs for ms_level=2
DEBUG  : PeakSampler                    : Retrieving mz_intensity_rt values from <vimms.DataGenerator.DataSource object at 0x00000219E68862E8>
INFO   : DataSource                     : Using values from scans


Saving <class 'vimms.DataGenerator.PeakSampler'> to C:\Users\joewa\Work\git\vimms\examples\example_data\peak_sampler_mz_rt_int_19_beers_fullscan.p


In [14]:
ps.get_peak(1, 10) # try to sample 10 MS1 peaks

[Peak mz=436.5262 rt=239.19 intensity=628800.55 ms_level=1,
 Peak mz=155.6701 rt=876.33 intensity=872327.88 ms_level=1,
 Peak mz=333.1338 rt=641.82 intensity=296933.13 ms_level=1,
 Peak mz=196.1309 rt=541.60 intensity=218339.48 ms_level=1,
 Peak mz=184.7690 rt=459.74 intensity=1059487.94 ms_level=1,
 Peak mz=276.4655 rt=1091.89 intensity=226547.55 ms_level=1,
 Peak mz=220.6405 rt=574.79 intensity=881250.10 ms_level=1,
 Peak mz=236.0961 rt=270.49 intensity=2272904.96 ms_level=1,
 Peak mz=323.3278 rt=8.77 intensity=27572.38 ms_level=1,
 Peak mz=245.2653 rt=465.16 intensity=80725.96 ms_level=1]

### Load fragmentation data and train KDEs

In [15]:
mzml_path = Path(base_dir, 'beers', 'fragmentation', 'mzML')
xcms_output = Path(mzml_path, 'extracted_peaks_ms1.csv')
out_file = Path(base_dir, 'peak_sampler_mz_rt_int_19_beers_fragmentation.p')

In [16]:
ds_fragmentation = get_data_source(mzml_path, filename, xcms_output)

INFO   : DataSource                     : Loading Beer_multibeers_10_T10_POS.mzML
INFO   : DataSource                     : Loading Beer_multibeers_11_T10_POS.mzML
INFO   : DataSource                     : Loading Beer_multibeers_12_T10_POS.mzML
INFO   : DataSource                     : Loading Beer_multibeers_13_T10_POS.mzML
INFO   : DataSource                     : Loading Beer_multibeers_14_T10_POS.mzML
INFO   : DataSource                     : Loading Beer_multibeers_15_T10_POS.mzML
INFO   : DataSource                     : Loading Beer_multibeers_16_T10_POS.mzML
INFO   : DataSource                     : Loading Beer_multibeers_17_T10_POS.mzML
INFO   : DataSource                     : Loading Beer_multibeers_18_T10_POS.mzML
INFO   : DataSource                     : Loading Beer_multibeers_19_T10_POS.mzML
INFO   : DataSource                     : Loading Beer_multibeers_1_T10_POS.mzML
INFO   : DataSource                     : Loading Beer_multibeers_2_T10_POS.mzML
INFO   : DataSourc

In [17]:
ps = get_spectral_feature_database(ds_fragmentation, filename, min_ms1_intensity, min_ms2_intensity, min_rt, max_rt,
               bandwidth_mz_intensity_rt, bandwidth_n_peaks, out_file)

DEBUG  : PeakSampler                    : Extracted 138969 MS2 scans
DEBUG  : PeakSampler                    : Computing parent intensity proportions
DEBUG  : PeakSampler                    : Extracting scan durations
DEBUG  : PeakSampler                    : Training KDEs for ms_level=1
DEBUG  : PeakSampler                    : Retrieving mz_intensity_rt values from <vimms.DataGenerator.DataSource object at 0x00000219CF0A02B0>
INFO   : DataSource                     : Using values from XCMS peaklist
DEBUG  : PeakSampler                    : Retrieving n_peaks values from <vimms.DataGenerator.DataSource object at 0x00000219CF0A02B0>
DEBUG  : PeakSampler                    : Training KDEs for ms_level=2
DEBUG  : PeakSampler                    : Retrieving mz_intensity_rt values from <vimms.DataGenerator.DataSource object at 0x00000219CF0A02B0>
INFO   : DataSource                     : Using values from scans
DEBUG  : PeakSampler                    : Retrieving n_peaks values from <vimms

Saving <class 'vimms.DataGenerator.PeakSampler'> to C:\Users\joewa\Work\git\vimms\examples\example_data\peak_sampler_mz_rt_int_19_beers_fragmentation.p


In [18]:
ps.get_peak(1, 10)

[Peak mz=164.4641 rt=365.60 intensity=460716.54 ms_level=1,
 Peak mz=369.1188 rt=248.70 intensity=89880.65 ms_level=1,
 Peak mz=112.7248 rt=536.25 intensity=1438033.05 ms_level=1,
 Peak mz=192.5910 rt=1096.60 intensity=524107.90 ms_level=1,
 Peak mz=392.5192 rt=881.76 intensity=276855.94 ms_level=1,
 Peak mz=150.7410 rt=800.90 intensity=479561.73 ms_level=1,
 Peak mz=234.5093 rt=1052.08 intensity=6734.82 ms_level=1,
 Peak mz=187.4759 rt=308.83 intensity=57290.45 ms_level=1,
 Peak mz=468.8168 rt=7.53 intensity=23809.31 ms_level=1,
 Peak mz=300.0799 rt=767.14 intensity=132245.77 ms_level=1]

In [19]:
ps.get_peak(2, 10)

[Peak mz=147.4304 rt=751.24 intensity=355.63 ms_level=2,
 Peak mz=80.7752 rt=1044.33 intensity=2351.46 ms_level=2,
 Peak mz=110.0894 rt=824.11 intensity=165347.71 ms_level=2,
 Peak mz=84.4166 rt=790.98 intensity=118125.19 ms_level=2,
 Peak mz=264.5343 rt=804.87 intensity=52708.95 ms_level=2,
 Peak mz=147.3034 rt=390.23 intensity=935.73 ms_level=2,
 Peak mz=85.2104 rt=345.48 intensity=1243.76 ms_level=2,
 Peak mz=252.4904 rt=556.53 intensity=21982.15 ms_level=2,
 Peak mz=220.0599 rt=1148.39 intensity=18443.36 ms_level=2,
 Peak mz=87.1350 rt=772.49 intensity=48610.07 ms_level=2]

## d. Extract the ROIs for DsDA Experiments

In [20]:
roi_mz_tol = 10
roi_min_length = 2
roi_min_intensity = 1.75E5
roi_start_rt = min_rt
roi_stop_rt = max_rt

#### Extract beer ROIs

In [21]:
file_names = Path(base_dir, 'beers', 'fragmentation', 'mzML').glob('*.mzML')
out_dir = Path(base_dir,'DsDA', 'DsDA_Beer', 'beer_t10_simulator_files')
mzml_path = Path(base_dir, 'beers', 'fragmentation', 'mzML')

extract_roi(list(file_names), out_dir, 'beer_%d.p', mzml_path, ps)

DEBUG  : RoiToChemicalCreator           :      0/ 11925
INFO   : RoiToChemicalCreator           : Found 11925 ROIs above thresholds


Created C:\Users\joewa\Work\git\vimms\examples\example_data\DsDA\DsDA_Beer\beer_t10_simulator_files
Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\DsDA\DsDA_Beer\beer_t10_simulator_files\beer_10.p


DEBUG  : RoiToChemicalCreator           :      0/  9971
INFO   : RoiToChemicalCreator           : Found 9971 ROIs above thresholds


Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\DsDA\DsDA_Beer\beer_t10_simulator_files\beer_11.p


DEBUG  : RoiToChemicalCreator           :      0/ 11249
INFO   : RoiToChemicalCreator           : Found 11249 ROIs above thresholds


...

Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\DsDA\DsDA_Beer\beer_t10_simulator_files\beer_7.p


DEBUG  : RoiToChemicalCreator           :      0/ 13179
INFO   : RoiToChemicalCreator           : Found 13179 ROIs above thresholds


Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\DsDA\DsDA_Beer\beer_t10_simulator_files\beer_8.p


DEBUG  : RoiToChemicalCreator           :      0/ 14842
INFO   : RoiToChemicalCreator           : Found 14842 ROIs above thresholds


Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\DsDA\DsDA_Beer\beer_t10_simulator_files\beer_9.p


#### Extract urine ROIs

In [22]:
file_names = Path(base_dir, 'urines', 'fragmentation', 'mzML').glob('*.mzML')
out_dir = Path(base_dir,'DsDA', 'DsDA_Urine', 'urine_t10_simulator_files')
mzml_path = Path(base_dir, 'urines', 'fragmentation', 'mzML')

extract_roi(list(file_names), out_dir, 'urine_%d.p', mzml_path, ps)

DEBUG  : RoiToChemicalCreator           :      0/ 16321
INFO   : RoiToChemicalCreator           : Found 16321 ROIs above thresholds


Created C:\Users\joewa\Work\git\vimms\examples\example_data\DsDA\DsDA_Urine\urine_t10_simulator_files
Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\DsDA\DsDA_Urine\urine_t10_simulator_files\urine_2.p


DEBUG  : RoiToChemicalCreator           :      0/ 18395
INFO   : RoiToChemicalCreator           : Found 18395 ROIs above thresholds


Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\DsDA\DsDA_Urine\urine_t10_simulator_files\urine_3.p


DEBUG  : RoiToChemicalCreator           :      0/ 16100
INFO   : RoiToChemicalCreator           : Found 16100 ROIs above thresholds


...

Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\DsDA\DsDA_Urine\urine_t10_simulator_files\urine_85.p


DEBUG  : RoiToChemicalCreator           :      0/ 14048
INFO   : RoiToChemicalCreator           : Found 14048 ROIs above thresholds


Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\DsDA\DsDA_Urine\urine_t10_simulator_files\urine_93.p


DEBUG  : RoiToChemicalCreator           :      0/ 16320
INFO   : RoiToChemicalCreator           : Found 16320 ROIs above thresholds


Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\DsDA\DsDA_Urine\urine_t10_simulator_files\urine_97.p
